<a href="https://colab.research.google.com/github/SuperNZH/Deep-Learning-Practice/blob/main/Dive%20in%20DL/7.0_byModule.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

# 继承Module

In [2]:
class MLP(nn.Module):
  def __init__(self, **kwargs):
    super(MLP, self).__init__(**kwargs)
    self.hidden = nn.Linear(784, 256)
    self.act = nn.ReLU() # act 是激活函数的意思
    self.output = nn.Linear(256, 10)
  # 定义前向运算，根据x计算返回所需要的模型输出
  def forward(self, x):
    a = self.act(self.hidden(x))
    return self.output(a)

# 不需要定义反向函数，是因为这个class里面系统会自动求梯度，然后自动生成反向传播的backward函数

In [3]:
X = torch.rand(2,784)
print(X)
net = MLP()
print(net)
net(X)

tensor([[0.4609, 0.1118, 0.2783,  ..., 0.7517, 0.3530, 0.0044],
        [0.0986, 0.6646, 0.2299,  ..., 0.1502, 0.5865, 0.7586]])
MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0737, -0.0027, -0.0751, -0.0714,  0.1377,  0.1214, -0.2058, -0.0618,
          0.1190, -0.1268],
        [-0.0682, -0.1824, -0.0720, -0.1660,  0.0308,  0.0348,  0.0750, -0.0629,
          0.1925, -0.0004]], grad_fn=<AddmmBackward0>)

# Module子类 --> Sequential类

In [4]:
# 如果模型的前向计算为简单串联各个层的计算时，可以用Sequential来更简单的定义模型
# 这个是Sequential大概的运行逻辑
class MySequential(nn.Module):
  from collections import OrderedDict
  def __init__(self, *args):
    super(MySequential, self).__init__()
    if len(args) == 1 and isinstance(args[0], OrderedDict):
      for key, module in args[0].items():
        self.add_module(key, module)
    else:
      for idx, module in enumerate(args):
        self.add_module(str(idx), module)

  def forward(self, input):
    for module in self._modules.values():
      input = module(input)
    return input

In [5]:
net = MySequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1671, -0.3619,  0.1082,  0.2003, -0.0128, -0.1814,  0.2324, -0.1886,
          0.0399,  0.0446],
        [ 0.0598, -0.3004,  0.0802,  0.0998,  0.1987, -0.3012,  0.3091, -0.0998,
          0.0252, -0.1380]], grad_fn=<AddmmBackward0>)

# Module子类 --> ModuleList类

In [7]:
# ModuleList 接受一个子模块的列表作为输入，像list一样可以append和extend

net = nn.ModuleList([nn.Linear(784,256), nn.ReLU()])
net.append(nn.Linear(256,10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


# Module子类 --> ModuleDict类

In [12]:
net = nn.ModuleDict({
    'linear': nn.Linear(784,256),
    'act': nn.ReLU(),
})
net['output'] = nn.Linear(256,10)
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (linear): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)
